In [ ]:
from ipywidgets import Layout, Button, VBox, Label, HBox
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
from ipywidgets import Button, HBox, VBox, Layout , FileUpload
import numpy as np
import xml.etree.ElementTree as ET
from skimage import io, feature, morphology
from skimage import measure
from  scipy import ndimage as nd
from skimage.color import label2rgb
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.measure import label, regionprops, points_in_poly
from skimage import data, filters, measure, morphology
from PIL import Image
import base64
from io import BytesIO
import random
import string
import datetime
from scipy import interpolate


layout = Layout(overflow_x='hidden',
                    overflow_y='auto',
                    border='3px solid black',
                    width='152px',
                    height='35px',
                    flex_flow='column',
                    display='flex')

layoutfull = Layout(overflow_x='hidden',
                    overflow_y='auto',
                    border='3px solid black',
                    width='480px',
                    height='35px',
                    flex_flow='column',
                    display='flex')

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%')

app_layout = Layout(border='3px solid black',
                    width='480px'
                   )
gen_layout = Layout(width='480px'
                   )

path = "./DB_12215/"




In [ ]:
# Create global material table
materiauTable = pd.DataFrame(columns = ["Produit","Matiere","Reference","Masse volumique ρ","Épaisseur du stratifié par masse de fibre sèche t/w","Module d’élasticité E Ef1//Fibres","Module d’élasticité E Ef2⏊ Fibres","Module de cisaillement G","Coefficient de Poisson ν","contrainte de cisaillement interlaminaire τUIL","εut1","εut2","εuc1","εuc2","εuf1b","εuf2c","γu12","σut1","σut2","σuc1","σuc2","σuf1","σuf2","τ12"])

#read ISO11215-C.4
file = "table C.4.csv"
table = pd.read_csv(path + file)

#read ISO11215-C.6
file = "table C.6.csv"
table6 = pd.read_csv(path + file)



# filter renfort & matrices
liste_renfort = table[(table['Type'] == 'Fibres de renfort')]
liste_matrix = table[(table['Type'] == 'Matrices')]

In [ ]:
#Create code generator
def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [ ]:


# Built tip list
tip_renfort = "Masse volumique ρ (ρf ou ρm): " \
    +liste_renfort['Masse volumique ρ (ρf ou ρm)'] \
    + table.iloc[0,2] \
    + '\n' \
    +"Module d’élasticité E Ef1//Fibres: " \
    +liste_renfort['Module d’élasticité E Ef1//Fibres'] \
    + table.iloc[0,3] \
    + '\n-\n' \
    +liste_renfort['Reference'] 
tip_renfort = tip_renfort.tolist()

# Create renfort button
renfortButton = widgets.ToggleButtons(
    options=liste_renfort['Denomination'],
    description='Renforts:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=tip_renfort,
    layout=layout,
#     icons=['check'] * 3
)


In [ ]:
# Create matrix button
tip_matrix = "Masse volumique ρ (ρf ou ρm): " \
    +liste_matrix['Masse volumique ρ (ρf ou ρm)'] \
    + table.iloc[0,2] \
    + '\n' \
    +"Module d’élasticité E Ef1//Fibres: " \
    +liste_matrix['Module d’élasticité E Ef1//Fibres'] \
    + table.iloc[0,3] \
    + '\n-\n' \
    +liste_matrix['Reference'] 
tip_matrix = tip_matrix.tolist()
matrixButton = widgets.ToggleButtons(
    options=liste_matrix['Denomination'],
    description='Matrices:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=tip_matrix,
    layout=layout,
#     icons=['check'] * 3
)

In [ ]:
# Create phi button
phiButton = widgets.FloatSlider(
    value=7.5,
    min=0.05,
    max=0.95,
    step=0.001,
    description='Taux de fibre',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [ ]:
# create one pd dataframe with one column
options = pd.DataFrame(columns = ['Produit'])

# Feed data frame
options['Produit'] = ['Unidirectionnel\r«UD»' \
            ,'Mat de verre\r «CSM»\r' \
            ,'Roving tissé' \
            ,'Bidirectionnel 0/90°\r «BD+»' \
            ,'Double biais ±45\r «DBx»' \
            ,'Quadriaxial\r0/45/90/_45\r «Qx»']


# Create produit button
produitButton = widgets.ToggleButtons(
    options=options['Produit'],
    description='Produit:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['1','2','3','4','5','6','7'],
    layout=layout,
#     icons=['check'] * 3
)


In [ ]:
# Keep created material
keepButton = widgets.Button(
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Conserve le materiau cree',
    icon='check'
)

In [ ]:
# Calcul des proprietes mecaniques des produit
def calc_mech_prop(renfort,matrix,produit,compound,value):
    
    # Formatage des entrees
    Ef1 = np.float(renfort[1]) 
    Ef2 = np.float(renfort[2]) 
    Em = np.float(matrix[1])
    Gf = np.float(renfort[3])
    Gm = np.float(matrix[3])
    nuf = np.float(renfort[4])
    num = np.float(matrix[4])
    rhof = np.float(renfort[0])
    rhom = np.float(matrix[0])
    epsilonUt1 = np.float(compound.iloc[0,2])
    epsilonUt2 = np.float(compound.iloc[0,3])
    epsilonUc1 = np.float(compound.iloc[0,4])
    epsilonUc2 = np.float(compound.iloc[0,5])
    epsilonUf1 = np.float(compound.iloc[0,6])
    epsilonUf2 = np.float(compound.iloc[0,7])
    tauU12 = np.float(compound.iloc[0,8])
    
    
    #Calcul de tsurw selon le choix unite + forcage tvf
    unite = 'tvf'
    if unite == 'tmf':
        phi = value/(value+(1-value)*rhof/rhom)
    else:
        phi = value
    tsurw = 1/phi/rhof
    rhoc = phi*rhof + (1-phi)*rhom
    
    #Calcul de la valeur maximum de cisaillement interlaminaire
    tauUIL = 22.5-33*phi/(phi+0.89)
    
    EUD1 = 0.975*(Ef1*phi+Em*(1-phi))
    etaE = (Ef2/Em-1)/(Ef2/Em+1)
    EUD2 = Em*(1+etaE*phi)/(1-etaE*phi)
    etaG = (Gf/Gm-1)/(Gf/Gm+1)
    GUD = Gm*(1+etaG*phi)/(1-etaG*phi)
    nuUD12 = nuf*phi+num*(1-phi)
    nuUD21 = nuUD12*EUD2/EUD1
    #return EUD1,EUD2,GUD,nuUD12,nuUD21,tauUIL

    #calc_mech_prop_CSM(EUD1,EUD2):
    ECSM = 3/8*EUD1+5/8*EUD2
    GCSM = 1/8*EUD1-1/4*EUD2
    # QuestionGregoire nuCSM tableauC.5 formuleisOK ?
    nuCSM = ECSM/2/(GCSM-1)
    #return ECSM, GCSM, nuCSM

    # QuestionGregoire GUD12 tableauC.5 correcttypois GUD12 or GUD ?
    #calc_mech_prop_BDP(EUD1,EUD2,GUD):
    EBDP = 0.5*(EUD1 + EUD2)
    GBDP = GUD
    nuBDP = nuUD12*EUD2/EBDP
    #return EBDP,GBDP,nuBDP

    #calc_mech_prop_DBX(EBDP,GBDP,nuBDP):
    EDBX = 4*EBDP/(EBDP/GBDP+2*(1-nuBDP))
    GDBX = EBDP/(1+nuBDP)/2
    nuDBX = EDBX/4/EBDP*(EBDP/GBDP-2*(1-nuBDP))
    #return EDBX, GDBX, nuDBX

    #calc_mech_prop_QDX(EBDP,EDBX,GBDP,GDBX,nuBDP,nuDBX):
    A11 = EBDP/(1-nuBDP**2)+EDBX/(1-nuDBX**2)
    A12 = nuBDP*EBDP/(1-nuBDP**2)+nuDBX*EDBX/(1-nuDBX**2)
    EQDX = 0.5*(A11-A12**2/A11)
    GQDX = 0.5*(GBDP + GDBX)
    nuQDX = A12/A11
    #return EQDX,GQDX,nuQDX
    
    if produit == 'Unidirectionnel\r«UD»':
        
        E1 = EUD1
        E2 = EUD2
        G12 = GUD
        nu12 = nuUD12
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Mat de verre\r «CSM»\r':
        
        E1 = ECSM
        E2 = ECSM
        G12 = GCSM
        nu12 = nuCSM
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Roving tissé':
        
        E1 = EBDP
        E2 = EBDP
        G12 = GBDP
        nu12 = nuBDP
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
    
    elif produit == 'Bidirectionnel 0/90°\r «BD+»':
        
        E1 = EBDP
        E2 = EBDP
        G12 = GBDP
        nu12 = nuBDP
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Double biais ±45\r «DBx»':
        
        E1 = EDBX
        E2 = EDBX
        G12 = GDBX
        nu12 = nuDBX
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
    elif produit == 'Quadriaxial\r0/45/90/_45\r «Qx»':
        
        E1 = EQDX
        E2 = EQDX
        G12 = GQDX
        nu12 = nuQDX  
        sigmaUt1 = E1*epsilonUt1 
        sigmaUt2 = E2*epsilonUt2
        sigmaUc1 = E1*epsilonUc1
        sigmaUc2 = E2*epsilonUc2
        sigmaUf1 = E1*epsilonUf1
        sigmaUf2 = E2*epsilonUf2
        tauU12 = G12*tauU12
     
        return [rhoc, tsurw,E1,E2,G12,nu12,tauUIL,sigmaUt1,sigmaUt2,sigmaUc1,sigmaUc2,sigmaUf1,sigmaUf2,tauU12]
        
        
    

In [ ]:
def on_value_change(change):
    global table
    global table6
    global materiauTable
    # extraction caracteristiques renfort
    renfortchoix = renfortButton.value
    extract = table[(table['Denomination'] == renfortchoix)]
    renfort = extract.iloc[0,3:8].tolist()
    # extraction caracteristiques matrices
    matrixchoix = matrixButton.value
    extract = table[(table['Denomination'] == matrixchoix)]
    matrix = extract.iloc[0,3:8].tolist()
    # already a float
    produit = produitButton.value
    # already a float
    value = phiButton.value
    # extraction des valeurs de deformations ultimes
    compound = table6[(table6['Produit'] == produit)&(table6['Matiere'] == renfortchoix + ' et ' + matrixchoix)]
    #epsilonUt1 = np.float(compound.iloc[0,2])

    #Execute kernel
    res = calc_mech_prop(renfort,matrix,produit,compound,value)

    #Dynamic plot update
    with a.batch_update():
        a.data[0].x=[np.float(res[1])]
        b.data[0].x=[np.float(res[2])]
        c.data[0].x=[np.float(res[9])]
        d.data[0].x=[np.float(res[13])]
        
renfortButton.observe(on_value_change, names='value')
matrixButton.observe(on_value_change, names='value')
produitButton.observe(on_value_change, names='value')
phiButton.observe(on_value_change, names='value')


In [ ]:
# General datas

In [ ]:
#Boat parameters importation
#tree = ET.parse('mach40.iscn')
#root = tree.getroot()
#placeholder  = [widgets.HTML(root[i].tag) for i in [9,12]]
#value = [widgets.Text(root[i].text) for i in [9,12]]
#left = widgets.VBox(placeholder)
#right = widgets.VBox(value)
#boatParameterMenu1 = widgets.HBox([left,right],layout={'width': 'max-content'})



# Create parameter slider
lwlButton = widgets.FloatSlider(
    value=2.500,
    min=2.5,
    max=24.0,
    step=0.001,
    description='Lwl:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

# Create parameter slider
mldcButton = widgets.FloatSlider(
    value=10000,
    min=10,
    max=100000,
    step=1,
    description='mLDC:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

# Create parameter slider
speedButton = widgets.FloatSlider(
    value=1,
    min=0,
    max=100,
    step=1,
    description='V:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)


In [ ]:
#Boat Characteristics input section components

boatcharacteristics1 = widgets.HTML(     "<font size=5 color=#b39732>DESCRIBE YOUR BOAT</font><br>"+
                                         "<font size=3>Be assisted by our interactive user entry that will give a instantaneous and complete insight of the standard, and compare your values to more than a thousand existing boats already fully EC-certified as productions or custom boats."+
                                         "<br>",layout=gen_layout
                                        )
boatcharacteristics2 = widgets.HTML("<font size=3><br>Beyong these bounds, if your boat project is in the scope of the standard, as given in <font size=1>ISO12215-5 1 Scope</font>, the standard should apply taking also into considerations extra analysis that should be done by other described methods.</font>",layout=gen_layout)
boatcharacteristics3 = widgets.HTML("<font size=3><br>▌Displacement of the boat in kg</font>,<br><font size=2> including all appendages, in fully loaded condition, should be given according to <font size=1>ISO 8666 Small craft — Principal data</font><font size=2>. The maximum and minimum values of the displacement slider are automatically adjusted based on the displacement in D category of 925 sailing boats and 276 motor boats including semi-rigids.</font>",layout=gen_layout )
boatcharacteristics4 = widgets.HTML("<font size=3><br>▌Lenght of the waterline in meters</font>,<br><font size=2> boat upright in fully loaded condition should be given according to <font size=1>ISO12215-5 3 Terms and definitions -Table 1.</font>",layout=gen_layout )

boatcharacteristics5 = widgets.HTML("<font size=1>Scroll and click to valid your choice</font>"
                                        )

boatcharacteristics6 = widgets.HTML("<font size=1>User the slider or enter directly the value on the right to enter the value</font>"
                                        )
boatcharacteristics7 = widgets.HTML("<font size=3><br>▌Design Category</font>,<br><font size=2> should be given according to the European Recreational Craft Directive 2013/53/EU <font size=1>Annex 1 - Essential requirements.</font>",layout=gen_layout )

boatcharacteristics8 = widgets.HTML("<font size=3><br>▌General type</font>,<br><font size=2> Choose from pre-segmented categories of products that reflects the market as analysed in 2020 by the Federation des Industries Nautiques in his annual report.</font>",layout=gen_layout )
boatcharacteristics9 = widgets.HTML("<font size=3><br>▌Performances</font>,<br><font size=2> Maximum speed in fully loaded conditions, in kts should be given according to <font size=1>ISO12215-5 3 Terms and definitions -Table 1.</font>",layout=gen_layout )

boatcharacteristics10 = widgets.HTML("Nothing to display yet.",layout=app_layout)
boatcharacteristics11 = widgets.HTML("Nothing to display yet.",layout=app_layout)


boatcharacteristics12 = widgets.HTML("<font size=3><br>▌Usage</font><br><font size=2> The new feature of the standard makes mandatory to define the usage of the boat. Insights into the  new regulatory  framework and its application to vessel in professional use.</font>",layout=gen_layout )
boatcharacteristics13 = widgets.HTML("Nothing to display yet.",layout=app_layout)

boatcharacteristics14 = widgets.HTML("<font size=3><br>▌Position on market</font>,<br><font size=2> Get a extra sense of your position on the market</font>",layout=gen_layout )

boatcharacteristics15 = widgets.HTML("<font size=1>Query when all informations above have been filled</font>"
                                    )

def signal_modifications_BoatOnMarket(self):
    boatcharacteristics15.value = "<font size=1>Informations above has been changed. Make a new query</font>"
                                    




In [ ]:
#Building

buildingcharacteristics1 = widgets.HTML("<br>"+"<br>"+"<br>"
                                         "<font size=5 color=#b39732>EXPLAIN HOW SHE IS BUILD</font><br>"+
                                         "<font size=3>Use other materials, including new fibres and resins, provided that they show similar cohesion, durability, resistance to marine environment and elongation at break as the ones quoted in the standard."+
                                         "<br>",layout=gen_layout
                                        )

buildingcharacteristics2 = widgets.HTML("<br><font size=3>▌Building material</font>,<font size=2> should be given according to <font size=1>ISO12215-5 5.1 Materials .</font>The materials available are taken according to  <font size=1>ISO12215-5 Table 17</font>. ",layout=gen_layout )

buildingcharacteristics3 = widgets.HTML("<br><font size=3>▌Manufacturing process</font>,<font size=2> should be given according to <font size=1>ISO12215-5 Table 14.</font>. The usage of the parameter has been extended to metallic and wooden constructions.",layout=gen_layout )

buildingcharacteristics4 = widgets.HTML("<br><font size=3>▌Material quality</font>,<font size=2> should be given according to <font size=1>ISO12215-5 5.1 Table 15.</font> The usage of the parameter has been extended to metallic and wooden constructions.",layout=gen_layout )

buildingcharacteristics5 = widgets.HTML("Nothing to display yet.",layout=app_layout)

#read csv
file = "listeMateriau.csv"
listeTypeBateau = pd.read_csv(path + file)


# Create listeTypeBateau button
listeMateriau = widgets.ToggleButtons(
    options=listeTypeBateau['Matériau'],
    description='listeMateriau:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "480px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

#read csv
file = "listeProcede_de_stratification.csv"
listeProcede = pd.read_csv(path + file)


# Create listeTypeBateau button
listeProcedeButton = widgets.ToggleButtons(
    options=listeProcede['Procédé de fabrication'],
    description='listeProcess:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "480px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

#read csv
file = "listeQualite_du_materiau.csv"
listeQualite = pd.read_csv(path + file)


# Create listeTypeBateau button
listeQualiteButton = widgets.ToggleButtons(
    options=listeQualite['Qualité du matériau'],
    description='listeQualite:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "480px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

#read csv
file = "Tableau 14.csv"
listeBuildingQualite = pd.read_csv(path + file)

def update_qualite(self):
    global listeBuildingQualite
    if (listeQualiteButton.value != 'Mesurée')&(listeProcedeButton.value != "Chaudronnerie, soudure"):
        aaa = listeBuildingQualite[(listeBuildingQualite['Qualité du matériau']==listeQualiteButton.value)&(listeBuildingQualite['Procédé de fabrication']==listeProcedeButton.value)]
    else:
        aaa = listeBuildingQualite[(listeBuildingQualite['Qualité du matériau']== 'Mesurée')&(listeBuildingQualite['Procédé de fabrication']==listeProcedeButton.value)]
        aaa['Quality and control requirements'].values[0] = "Mechanical properties of materials are determined on test of the mechanical characteristics (modulus, breaking strains, etc.). Monitoring records, material batch traceability, frequency (once a year as a minimum) and test procedures are drawn up into a quality procedure, such as ISO 9001 or equivalent."    
    buildingcharacteristics5.value = "<font size=3>► General requirement</font><br><font size=2>The person or company that endorse the responsability of the boat construction should demonstrate that the following quality requirements are actually has properly followed the state of the art in terms of building technics,  <br>—environment (e.g. temperature, hygrometry, etc. during storage and building) <br>— building process (e.g. preparation process prior to building such as dusting, degreasing, priming, etc.)<br>— type of material (e.g. proper combination of material, other relevant) :<br><font size=3>► Quality and control</font><br>  "+aaa['Quality and control requirements'].values[0]+".</font>"
    
listeQualiteButton.observe(update_qualite)
listeProcedeButton.observe(update_qualite)




In [ ]:
#read csv
file = "listeTypeBateau.csv"
listeTypeBateau = pd.read_csv(path + file)


# Create listeTypeBateau button
listeTypeBateauButton = widgets.ToggleButtons(
    options=listeTypeBateau['Type'],
    description='listeTypeBateau:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "480px"},
    #tooltips='',
    layout=layoutfull,
#     icons=['check'] * 3
)

# create one pd dataframe with one column
listeDesignCategory = pd.DataFrame(columns = ['Catégorie de navigation'])


# Feed data frame

#read csv
file = "listeDesignCategory.csv"
listeDesignCategory = pd.read_csv(path + file,delimiter=",")


# Create design category button
listeDesignCategoryButton = widgets.ToggleButtons(
    options=listeDesignCategory['Design category'],
    description='Design category:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "90px"},
    #tooltips='',
    layout=layoutfull,
#      icons=['check'] * 3
)               


def update_TypeBateau(self):
    global listeTypeBateau
    global value
    propulsion = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,11]
    value[0].value = propulsion

    
def update_DesignCategory(self):
    global value
    value[1].value = listeDesignCategoryButton.value

listeTypeBateauButton.observe(update_TypeBateau)
listeDesignCategoryButton.observe(update_DesignCategory)


In [ ]:
# Feed data frame

#read csv
file = "table J.1.csv"
listeUsage = pd.read_csv(path + file,delimiter=";")

# Create usagecategory button
listeUsageButton = widgets.ToggleButtons(
    options=listeUsage['Usage'],
    description='Usage:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={"button_width": "480px"},
    #tooltips='',
    layout=layoutfull,
#      icons=['check'] * 3
)               
               

# create design category table
trace55 = go.Table(
    header=dict(values=['Restriction on usage','Operating conditions'],
                fill_color='#b39732',
                line_color='white',
                font=dict(color='white'),
                #align=['left','left','left','center','center','center'],
               ),
    cells=dict(#values=materiauTable.transpose().values.tolist(),
                values = ['',''],
                fill_color='#3ba7b3',
                line_color='white',
                font=dict(color='white'),
                #align=['left','left','left','center','center','center'],
                #format = [None,None,None,".3f",".3f",".0f"]),
                ))
layout55 = go.Layout(title='',
                    width=480,
                    height=155,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=5,
                                pad=4
                                ),
                   )
usagetable55 = go.FigureWidget(data=[trace55], layout=layout55)




def update_category(self):
    global listeUsage
    aaa = listeUsage[listeUsage['Usage']==listeUsageButton.value]
    usagetable55.update_traces(cells_values=aaa[['Restriction on usage','Operating conditions']].transpose().values.tolist(), selector=dict(type='table'))
    boatcharacteristics13.value = "<font size=2>► The user is responsible for prudent operation according to the prevailing conditions (wind and waves) according to the principles of good seamanship and recognizing the category for which the craft is designed.<br>► "+aaa['Maintenance'].values[0]+"<br>► "+aaa['Additional requirements'].values[0]+"</font>"   

    
    
listeUsageButton.observe(update_category)




In [ ]:
def update_minmax(self):
    global listeTypeBateau
    lwlButton.min = 0.0
    lwlButton.max = 25.00
    minnew = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,1]
    maxnew = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,2]
    lwlButton.min = minnew
    lwlButton.max = maxnew 
    return 

listeTypeBateauButton.observe(update_minmax)
listeTypeBateauButton.observe(signal_modifications_BoatOnMarket )   
 

In [ ]:
# upload image management

# create callback function for uploader            
def response(change):
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open('myfile.png', 'wb') as f: f.write(content)
    text = io.imread('myfile.png')
    
    pil_img = Image.fromarray(np.flipud(text)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
    with lol.batch_update():
        lol.data = []
        #details0.value = uploaded_filename+" has been uploaded."
        lol.add_trace(go.Image(source = base64_string))
        details0.value = uploaded_filename+" has been uploaded."
    with lol.batch_update():
        lol33.data = []
        lol33.update_annotations(visible=False)
        
uploader = FileUpload(multiple=False,description='Upload image')
uploader.observe(response, names='value')
        
# create root figure
layout6 = go.Layout(title='',
                    #width=480,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=5,
                                pad=4
                                ),
                   )



lol = go.FigureWidget(layout=layout6)

# define root figure layout

lol.update_xaxes(autorange=False)
lol.update_yaxes(autorange=False)
lol.update_xaxes(visible=False)  
lol.update_yaxes(visible=False)
lol.layout.hovermode = 'closest'
lol.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol.layout.showlegend = False
lol.layout.plot_bgcolor = 'white'
#lol.layout.width=480






In [ ]:

# create root figure
layout11 = go.Layout(title='',
                    width=480,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=0,
                                pad=4
                                ),
                   )



lol11 = go.FigureWidget(layout=layout11)



# define root figure layout

lol11.update_xaxes(autorange=False)
lol11.update_yaxes(autorange=False)
lol11.update_xaxes(visible=False)  
lol11.update_yaxes(visible=False)


lol11.layout.hovermode = 'closest'
lol11.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol11.layout.showlegend = False
lol11.layout.plot_bgcolor = 'white'
lol11.layout.width=480
lol11.layout.height=720





def create_annexJ_image(self):
    global listeTypeBateau
    
    #get AnnexJ
    text = io.imread(path + 'annexLFULL.png')

    pil_img = Image.fromarray(np.flipud(text)) # PIL image object
    prefix = "data:image/png;base64,"
    with BytesIO() as stream:
        pil_img.save(stream, format="png")
        base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")

    lol11.data = []
    lol11.update_annotations(visible=False)
    
    # add trace
    lol11.add_trace(go.Image(source = base64_string))

    #read csv
    file = "template_annexJ.csv"
    template_annexJ = pd.read_csv(path + file,delimiter=",")



    if 'monocoque' in listeTypeBateauButton.value:
        res = template_annexJ[template_annexJ['name'] == 'monohull']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                       y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                       text = 'x',
                       showarrow = False,
                       yshift=0)
    elif 'multicoque' in listeTypeBateauButton.value:
        res = template_annexJ[template_annexJ['name'] == 'multihull']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                       y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                       text = 'x',
                       showarrow = False,
                       yshift=0)

    res2 = navmode(float(speedButton.value),float(lwlButton.value))


    propulsion = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,11]

    if propulsion == 'Motor':

        if res2[1] == 'Displacement':
            res = template_annexJ[template_annexJ['name'] == 'displacement']
            lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                           y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                           text = 'x',
                           showarrow = False,
                           yshift=0)

        elif res2[1] == 'Planning':
            res = template_annexJ[template_annexJ['name'] == 'planning']
            lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                           y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                           text = 'x',
                           showarrow = False,
                           yshift=0)

    if propulsion == 'Sailing' and (listeDesignCategoryButton.value == 'A' or listeDesignCategoryButton.value == 'B') :
            res3 = islightsailboat(float(mldcButton.value),float(lwlButton.value))
    else :
            res3=''

    if res3 == 'Light sailing boat' :
        res = template_annexJ[template_annexJ['name'] == 'kslsup']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)
    elif res3 == '':
        res = template_annexJ[template_annexJ['name'] == 'ksls']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)


    if listeMateriau.value == 'steel alloys':
        res = template_annexJ[template_annexJ['name'] == 'steel']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)


    if listeMateriau.value == 'aluminium alloys':
        res = template_annexJ[template_annexJ['name'] == 'aluminium']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)

    if listeMateriau.value == 'wood or plywood (single skin or sandwich)*':
        res = template_annexJ[template_annexJ['name'] == 'wood']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)


    if listeMateriau.value == 'fibre-reinforced plastics*':
        res = template_annexJ[template_annexJ['name'] == 'frp']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)

    if listeMateriau.value == 'non-reinforced plastics*':
        res = template_annexJ[template_annexJ['name'] == 'frp']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                        y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                        text = 'x',
                        showarrow = False,
                        yshift=0)

    res = template_annexJ[template_annexJ['name'] == 'lwl']
    lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = str('{:.3f}'.format(lwlButton.value)),
                    showarrow = False,
                    yshift=0)

    res = template_annexJ[template_annexJ['name'] == 'mldc']
    lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = str('{:.0f}'.format(mldcButton.value)),
                    showarrow = False,
                    yshift=0)

    res = template_annexJ[template_annexJ['name'] == 'v']
    lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = str('{:.1f}'.format(speedButton.value)),
                    showarrow = False,
                    yshift=0)

    if listeDesignCategoryButton.value == 'A':
        res = template_annexJ[template_annexJ['name'] == 'catA']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeDesignCategoryButton.value == 'B':
        res = template_annexJ[template_annexJ['name'] == 'catB']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeDesignCategoryButton.value == 'C':
        res = template_annexJ[template_annexJ['name'] == 'catC']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeDesignCategoryButton.value == 'D':
        res = template_annexJ[template_annexJ['name'] == 'catD']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeUsageButton.value == 'Private use' or listeUsageButton.value == 'Rental' or listeUsageButton.value == 'Charter'  :
        res = template_annexJ[template_annexJ['name'] == 'recrecharter']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeUsageButton.value == 'Light duty workboat' :
        res = template_annexJ[template_annexJ['name'] == 'workboatlight']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeUsageButton.value == 'Heavy duty workboat' :
        res = template_annexJ[template_annexJ['name'] == 'workboatheavy']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeQualiteButton.value == 'Mesurée' :
        res = template_annexJ[template_annexJ['name'] == 'tested']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeQualiteButton.value == 'Haute' :
        res = template_annexJ[template_annexJ['name'] == 'high']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    if listeQualiteButton.value == 'Basse' :
        res = template_annexJ[template_annexJ['name'] == 'low']
        lol11.add_annotation(x =(res['1stcornerX'].values[0]+res['2ndcornerX'].values[0])/2 ,
                    y =(res['1stcornerY'].values[0]+res['2ndcornerY'].values[0])/2 ,
                    text = 'x',
                    showarrow = False,
                    yshift=0)

    AnnexL4.value = "<font size=1>Formular is fullfilled according to the informations given above</font>"
    AnnexL3.value = "Form number <font size=1>ANF-" + get_random_string(8)+ "</font>      Date and time <font size=1>" + datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")+ "</font>" 
    #lol11.write_image('AnnexL_to_be_completed.pdf', engine="kaleido")


# create button for active action
create_annexJ_imageButton = widgets.Button(
    description='Get it',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Push to generate the content',
    icon='rectangle-list' # (FontAwesome names without the `fa-` prefix),
)

create_annexJ_imageButton.on_click(create_annexJ_image)

In [ ]:
#Annex L

AnnexL1 = widgets.HTML("<br>"+"<br>"+"<br>"
                     "<font size=5 color=#b39732>GET YOUR APPLICATION FORM</font><br>"+
                     "<font size=3>All the entries and output variables are translated into simple formulars, delivering a pre-organized and comprehensive information ready to share."+
                     "<br>",layout=gen_layout
                    )

AnnexL2 = widgets.HTML("<br><font size=3>▌Application form of ISO 12215-5</font>,<font size=2> According to <font size=1>ISO12215-5 Annex L",layout=gen_layout )


AnnexL3 = widgets.HTML("Nothing to display yet.",layout=app_layout)


AnnexL4 = widgets.HTML("<font size=1>Query when all informations above have been filled</font>"
                                    )

def signal_modifications_global(self):
    AnnexL4.value = "<font size=1>Informations above has been changed. Make a new query</font>"
    
    
listeQualiteButton.observe(signal_modifications_global)
listeProcedeButton.observe(signal_modifications_global)
listeUsageButton.observe(signal_modifications_global)
listeTypeBateauButton.observe(signal_modifications_global)
listeDesignCategoryButton.observe(signal_modifications_global)


In [ ]:
# create design category table
trace5 = go.Table(
    header=dict(values=['Wind force in Beaufort','Significant wave height H ⅓ in metres'],
                fill_color='#b39732',
                line_color='white',
                font=dict(color='white'),
                #align=['left','left','left','center','center','center'],
               ),
    cells=dict(#values=materiauTable.transpose().values.tolist(),
                values = ['',''],
                fill_color='#3ba7b3',
                line_color='white',
                font=dict(color='white'),
                #align=['left','left','left','center','center','center'],
                #format = [None,None,None,".3f",".3f",".0f"]),
                ))
layout5 = go.Layout(title='',
                    width=480,
                    height=125,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=5,
                                pad=4
                                ),
                   )
categorytable = go.FigureWidget(data=[trace5], layout=layout5)


def update_category(self):
    global listeDesignCategory
    aaa = listeDesignCategory[listeDesignCategory['Design category']==listeDesignCategoryButton.value]
    categorytable.update_traces(cells_values=aaa[['Wind force in Beaufort','Significant wave height H ⅓ in metres']].transpose().values.tolist(), selector=dict(type='table'))
    boatcharacteristics10.value = "<font size=2>► "+aaa['notes'].values[0]+"<br>► The boat should be designed and constructed to withstand the wind speed and sea state ranges given below while maintaining the manoeuvrability.</font>"
    
listeDesignCategoryButton.observe(update_category)
listeDesignCategoryButton.observe(signal_modifications_global) 

In [ ]:
def lwl2mldc(lwl,a,b,c,d):
    res = d + (a - d)/(1 + (lwl/b)**c)
    return res

# create root figure
layout7 = go.Layout(title='',
                    height=300,
                    width=480,
                    margin=dict(
                                l=5,
                                r=5,
                                b=5,
                                t=75,
                                pad=4
                                ),
                   )




lol2 = go.FigureWidget(layout=layout7)

def update_graph(self):
    global listeTypeBateau
    lol2.data = []
    aum = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,3]
    bum = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,4]
    cum = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,5]
    dum = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,6]
    alm = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,7]
    blm = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,8]
    clm = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,9]
    dlm = listeTypeBateau[listeTypeBateau['Type'] == 'Bateaux a moteurs habitables L>15m'].iloc[0,10]

    auv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,3]
    buv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,4]
    cuv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,5]
    duv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,6]
    alv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,7]
    blv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,8]
    clv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,9]
    dlv = listeTypeBateau[listeTypeBateau['Type'] == 'Voiles habitables monocoques L>15m'].iloc[0,10]

    t=np.linspace(2.5,24,50)

    if listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value]['Propulsion'].values[0] == 'Sailing':
        lol2.update_layout(
            title="EC certified sailing crafts market bounds, all times",
                )
        lol2.add_trace(go.Scatter(x=t,y=lwl2mldc(t,auv,buv,cuv,duv),mode='lines',line_color='#b3444d', line_width=4,name="Upper bound"))
        lol2.add_trace(go.Scatter(x=t,y=lwl2mldc(t,alv,blv,clv,dlv),mode='lines',line_color='#f3efe0', line_width=4,name="Lower bound"))
        
    else:
        lol2.update_layout(
            title="EC certified motorcrafts market bounds, all times",
                )

        lol2.add_trace(go.Scatter(x=t,y=lwl2mldc(t,aum,bum,cum,dum),mode='lines',line_color='#b3444d', line_width=4,name="Upper bound"))
        lol2.add_trace(go.Scatter(x=t,y=lwl2mldc(t,alm,blm,clm,dlm),mode='lines',line_color='#f3efe0', line_width=4,name="Lower bound"))
    
    
listeTypeBateauButton.observe(update_graph)
listeTypeBateauButton.observe(signal_modifications_BoatOnMarket )
listeTypeBateauButton.observe(signal_modifications_global) 

In [ ]:
def locateBoatOnMarket(self):
    lol2.update_annotations(visible=False)
    lol2.add_annotation(
        x=lwlButton.value,
        y=mldcButton.value,
        xref="x",
        yref="y",
        text="Position on market",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
    boatcharacteristics15.value = "<font size=1>Position is given according to the informations given above</font>"
                                    

# create button for active action
locateBoatOnMarketButton = widgets.Button(
    description='Get it',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Push to generate the content',
    icon='rectangle-list' # (FontAwesome names without the `fa-` prefix),
)

locateBoatOnMarketButton.on_click(locateBoatOnMarket)


In [ ]:
# fonction d'analyse bateau

def islightsailboat(mldc,lwl):
    if mldc<5*lwl**3:
        res = 'light sailing boat'
    else:
            res=''
    return res

def navprog(mldc,lwl):
    res = mldc/lwl**2.4
    if res < 12.5:
        prog='Extreme'
    elif (res >=12.5)&(res <14):
        prog='Racing'
    elif (res >=14.0)&(res <27.0):
        prog='Gran Turismo'
    elif (res >=27.0):
        prog='Grand cruiser'
    return res,prog
    
def navmode(V,lwl):
    res = V/lwl**.5
    if res >= 5:
        mode='Planning'
    else:
        mode='Displacement'
    return res,mode

def mldc2as(mldc):
    res = 0.07*mldc**(2/3)
    return res

In [ ]:
def lwl2mldc(lwl,a,b,c,d):
    res = d + (a - d)/(1 + (lwl/b)**c)
    return res

def update_mldc(self):
    global listeTypeBateau
    global value
    mldcButton.min = 0.0
    mldcButton.max = 100000
    au = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,3]
    bu = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,4]
    cu = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,5]
    du = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,6]
    al = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,7]
    bl = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,8]
    cl = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,9]
    dl = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,10]
    #trick to correct functionnality for very small unit.
    mldcButton.min = min(lwl2mldc(lwlButton.value,al,bl,cl,dl),lwl2mldc(lwlButton.value,au,bu,cu,du))
    mldcButton.max = max(lwl2mldc(lwlButton.value,al,bl,cl,dl),lwl2mldc(lwlButton.value,au,bu,cu,du))
    

def analyse_description(self):
    global listeTypeBateau
    global value
    propulsion = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,11]
    res1 = navprog(float(mldcButton.value),float(lwlButton.value))
    res2 = navmode(float(speedButton.value),float(lwlButton.value))
    if propulsion == 'Sailing' and (listeDesignCategoryButton.value == 'A' or listeDesignCategoryButton.value == 'B') :
        res3 = islightsailboat(float(mldcButton.value),float(lwlButton.value))
    else :
        res3=''
    
    As = mldc2as(float(mldcButton.value))
    Astext = str('{:.1f}'.format(As))+'m2'
    propulsiontext0 = "<font size=3>►"+propulsion + "</font> Principal mean of propulsion.<br>"
    progtext0 = "<font size=3>►"+res1[1] + "</font> Navigation programm of the boat.<br>"
    modetext0 = "<font size=3>►"+res2[1] + "</font> Mode of navigation.<br>"
    
    if propulsion == 'Sailing':
               propulsiontext1 = ". The total upwind sail area of the boat should be greater than "+Astext+" ,according to <font size=1>ISO12215-5 3</font>. If it is not the case, change the boat type to motorboat."
    
    elif propulsion == 'Motor':
             propulsiontext1=". However, if your boat is equipped with a sailing rig, the total surface of the boat should be lesser than "+Astext+". If it is not the case, your boat should be considered as a sailing boat, according to <font size=1>ISO12215-5 3</font>"
    
    if res2[1] == 'Displacement':
              modetext1 = "In accordance with <font size=1>ISO12215-5 3</font>, her weight should be mainly supported by buoyancy forces."
    
    elif res2[1] == 'Planning':
              modetext1 = "In accordance with <font size=1>ISO12215-5 3</font>, her weight should be significantly supported by forces coming from dynamic lift due to speed in the water."
    
    
    if res3 != 'light sailing boat':
        boatcharacteristics11.value ="<font size=2>" + progtext0 +"A generic category that has been determined based on the ratio of the boat displacement to a power of the waterlenght, with no reference to the actual standard. Far from being an absolut truth, Use it to compare you boat with the existing boats we have in our database, except multihulls. </font>." +"<br>" \
                                     +"<font size=2>" + propulsiontext0 + propulsiontext1 +"<br></font>" \
                                     +"<font size=2>" + modetext0 + modetext1 +"</font>" +"<br>"
    else:
        islightboat0 = " In addition, the boat should be considered in the application of the standard as a -"+res3+", according to <font size=1>ISO12215-5 8.7</font>."
        boatcharacteristics11.value = "<font size=2>" + progtext0 +"A generic category that has been determined based on the ratio of the boat displacement to a power of the waterlenght, with no reference to the actual standard. Far from being an absolut truth, Use it to compare you boat with the existing boats we have in our database, except multihulls. </font>." + "<br>"\
                                     +"<font size=2>" + propulsiontext0 + propulsiontext1 +islightboat0+"<br></font>" \
                                     +"<font size=2>" + modetext0 + modetext1 +"</font>" +"<br>"
    
    
    #update all parameters
    value[4].value = res2[1]
    value[16].value = str(lwlButton.value)
    value[19].value = str(mldcButton.value)
    value[20].value = str(speedButton.value)
    return

    
lwlButton.observe(update_mldc)
mldcButton.observe(analyse_description)
speedButton.observe(analyse_description)
lwlButton.observe(analyse_description)

mldcButton.observe(signal_modifications_BoatOnMarket )
speedButton.observe(signal_modifications_BoatOnMarket)
lwlButton.observe(signal_modifications_BoatOnMarket)

mldcButton.observe(signal_modifications_global )
speedButton.observe(signal_modifications_global )
lwlButton.observe(signal_modifications_global )


In [ ]:
# Make the app


# prepare output logic
lol2.layout.hovermode = 'closest'
#fig2.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol2.layout.showlegend = False
lol2.layout.plot_bgcolor = '#3ba7b3'
lol2.update_xaxes(showline=True, linewidth=1, linecolor='white', gridcolor='white')
lol2.update_yaxes(showline=True, linewidth=1, linecolor='white', gridcolor='white')

lol2.update_layout(
    title="EC certified market bounds, all times",
    xaxis_title="Waterline lenght in meters",
    yaxis_title="Displacement in kilogramme",
    #legend_title="Legend Title",
    font=dict(
        #family="Courier New, monospace",
        size=10,
        color="black"
    )
)


# preparation
lol.update_xaxes(showticklabels=False)
lol.update_yaxes(showticklabels=False)

trace1 = go.Bar( x = [0.01],y = [""], orientation='h')
layout1 = go.Layout(title='Épaisseur du stratifié par Masse de fibre sèche t/w',
                   xaxis=dict(range=[0,3]),
                   xaxis_title="longueur en millimetres",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
a = go.FigureWidget(data=[trace1],layout=layout1)
trace2 = go.Bar( x = [0.01],y = [""], orientation='h')
layout2 = go.Layout(title="Module d’élasticité E Ef1//Fibres",
                   xaxis=dict(range=[0,1e5]),
                   xaxis_title="module en MPa",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
b = go.FigureWidget(data=[trace2],layout=layout2)
trace3 = go.Bar( x = [0.01],y = [""], orientation='h')
layout3 = go.Layout(title="σuc1",
                   xaxis=dict(range=[0,1e5]),
                   xaxis_title="contrainte en MPa",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
c = go.FigureWidget(data=[trace3],layout=layout3)
trace4 = go.Bar( x = [0.01],y = [""], orientation='h')
layout4 = go.Layout(title="τ12",
                   xaxis=dict(range=[0,1e5]),
                   xaxis_title="contrainte en MPa",
                   width = 550,
                   height = 200,
                   title_x = 0,
                   legend_title_font_size=8)
d = go.FigureWidget(data=[trace4],layout=layout4)

#k = HBox([listeTypeBateauButton,listeDesignCategoryButton],layout={'width': 'max-content'})
i1 = HBox(children=[a,b])
i2 = HBox(children=[c,d])
i = VBox([i1,i2])
h = HBox([produitButton,renfortButton,matrixButton])
g = HBox([phiButton,keepButton],layout=box_layout)
igh = VBox([h,g,i])
gendata = VBox(children=[boatcharacteristics1,
               uploader,
               lol,
               boatcharacteristics7,
               boatcharacteristics5,
               listeDesignCategoryButton,
               boatcharacteristics10,
               categorytable,
               boatcharacteristics12,
               boatcharacteristics5,
               listeUsageButton,
               boatcharacteristics13,
               usagetable55,
               boatcharacteristics8,
               boatcharacteristics5,
               listeTypeBateauButton,
               boatcharacteristics4,
               boatcharacteristics6,
               lwlButton, 
               boatcharacteristics3,
               boatcharacteristics6,
               mldcButton,
               boatcharacteristics9,
               boatcharacteristics6,
               speedButton,
               boatcharacteristics11,
               boatcharacteristics14,
               boatcharacteristics15,
               locateBoatOnMarketButton,
               lol2,
               #boatcharacteristics2,
               buildingcharacteristics1,
               buildingcharacteristics2,
               boatcharacteristics5,
               listeMateriau,
               buildingcharacteristics3,
               boatcharacteristics5,
               listeProcedeButton,
               buildingcharacteristics4,
               boatcharacteristics5,
               listeQualiteButton,
               buildingcharacteristics5,
               AnnexL1,
               AnnexL2,
               AnnexL4,
               create_annexJ_imageButton,
               AnnexL3,
               lol11,
               #i,
               #g,
               #h],
              ],
                layout=Layout(
                                display='flex',
                                flex_flow='column',
                                #border='solid 2px',
                                justify_items='flex-start',
                                align_items='stretch',
                                width='100%',
                           )
     )


In [ ]:
# Pression Methode 1

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io, feature, morphology
from skimage import measure
from  scipy import ndimage as nd
from skimage.color import label2rgb
from skimage.color import rgb2gray

import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
from skimage.measure import label, regionprops, points_in_poly,  subdivide_polygon, approximate_polygon, find_contours, regionprops_table

import plotly.express as px
import plotly.graph_objects as go
from skimage import data, filters, measure, morphology

from ipywidgets import HTML
from ipywidgets import Output, VBox
from ipywidgets import Button, HBox, VBox, Box, Layout, FileUpload
import ipywidgets as widgets

import xml.etree.ElementTree as ET

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, HTML
import plotly.graph_objs as go
import ipywidgets as widgets



In [ ]:
# Calcul selon la methode(1)

def methode1(zone,zone_detail,position,ZQ,ZSDA,ZC,el2,el3):
    global tree
    global root
    global listeTypeBateau
    
    ##Parametres bateau------------------------------------
    
    #affecte les entrees ISCANT
    #indexCategorie = root.find('./indexCategorie').text
    
    
    # on modifie dynamiquement la longueur LWL
    #LWL = float(root.find('./LWL').text)
    elm = root.find('./LWL')
    elm.text = lwlButton.value
    
    
    
    #BC = float(root.find('./BC').text)
    #D = float(root.find('./D').text)
    #beta04 = float(root.find('./beta').text)
  
    
    # on modifie dynamiquement mdlc
    #mLDC = float(root.find('./mLDC').text)
    elm = root.find('./mLDC')
    elm.text = mldcButton.value
    
    
    Gz = float(root.find('./Gz').text)
    
    # on modifie dynamiquement V
    #V = float(root.find('./V').text)
    elm = root.find('./V')
    elm.text = speedButton.value
    
    
    # on modifie dynamiquement la categorie de bateau
    elm = root.find('./indexCategorie')
    elm.text = listeDesignCategoryButton.value
    
    # on modifie dynamiquement le type de bateau
    elm = root.find('./indexType')
    elm.text = listeTypeBateau[listeTypeBateau['Type'] == listeTypeBateauButton.value].iloc[0,11]
    
    # on modifie dynamiquement le mode de bateau
    elm = root.find('./indexMode')
    res2 = navmode(float(speedButton.value),float(lwlButton.value))
    elm.text = res2[1]
    
    # on modifie dynamiquement la zone du bateau
    elm = root.find('./indexZone')
    elm.text = zoneButton.value
        
    # mistouflage du BC
    BWL = root.find('./BWL')
    BWL = BWL.text
    BC = root.find('./BC')
    BC.text = BWL

    # mistouflage du TC
    TC = root.find('./TC')
    TC.text = str(float(BWL)*0.35)

    
    #A MODIFIER affecte les entrees ajoutees
    BC = float(root.find('./BC').text)
    TC = float(root.find('./TC').text) 
    mLDC = float(root.find('./mLDC').text)
    BWL = float(root.find('./BWL').text)
    LWL = float(root.find('./LWL').text)
    V = float(root.find('./V').text)
    indexCategorie = root.find('./indexCategorie').text
    Type=root.find('./indexType').text
    Mode=root.find('./indexMode').text
    Zone=root.find('./indexZone').text
    ZoneDetail=root.find('./indexZoneDetail').text
    
    beta04 = 45
    
    # affecte la position de la zone
    x = float(position)
    ZQ = float(ZQ)
    ZSDA = float(ZSDA)
    #TC = float(tc)
    #TC = 2/5*D
    
    ##Parametres panneau--------------------------------------
    #el1 = root.findall("./Panneaux/Panneau/reference")[0].text
    #el2 = root.findall("./Panneaux/Panneau/l")[0].text
    #el3 = root.findall("./Panneaux/Panneau/b")[0].text
   
    l = float(el2)
    b = float(el3)

   
    # 8.2 Facteur de catégorie de conception kDC
    if indexCategorie == 'A':
        kDC = 1
    elif  indexCategorie == 'B':
        kDC = 0.8
    elif  indexCategorie == 'C':
        kDC = 0.6
    elif  indexCategorie == 'D':
        kDC = 0.4
        
    
    # 8.7 Facteur de correction de pression d’impact kSLS pour les voiliers stables et légers
    if indexCategorie == 'A' or indexCategorie == 'B':
        if mLDC>5*LWL**3:
            kSLS = 1
        elif mLDC<5*LWL**3:
            kSLS = max((10*Gz*LWL**0.5/mLDC**0.33)**0.5,1)
        elif  indexCategorie == 'C' or indexCategorie =='D':
            kSLS = 1
            
    
    # 8.3 Facteur de chargement dynamique kDYN
    kDYN1 = 0.32*(LWL/10/BC+0.084)*(50-beta04)*V**2*BC**2/mLDC
    kDYN2 = 0.5*V/mLDC**0.17
    
    if Mode == "Planning":
        kDYN = min(kDYN1,kDYN2)
    elif Mode == "Displacement":
        if kSLS > 1:
            kDYN = max(kDYN1,kDYN2,3)
        elif kSLS == 1:
            kDYN = 3
    
    
    # 8.5 Facteur de réduction de pression selon la surface kAR
    if Mode == "Planning":
        kR = 1
    else:
        if b >= 5000:
            kR = 1.0e-12
        else:
            kR = min(1.5-3e-4*b,1)
    AD=l*b*1e-6
    kAR = kR*0.1*mLDC**0.15/AD**.3
    
    
    # 8.4 Facteur de distribution longitudinale de pression kL
    kLa = 1.667-0.222*kDYN
    kLb = 0.133*kDYN
    kL = KL(x,kLa,kLb,LWL)
    
      
    # 8.6 Facteur de réduction de pression pour les superstructures ou les roufs kSUP
    if Zone == "Avant des superstructures":
        
        if ZoneDetail == "Zone ou on marche":
            kSUP = max(1-0.3*HSUP/cos(alphaL)/ZSDT(x,LWL),0.67)
        else:
            kSUP = max(1-0.3*HSUP/cos(alphaL)/ZSDT(x,LWL),0.5)
    
    elif Zone == "Flancs des superstructures":
         
        if ZoneDetail == "Zone ou on marche":
            kSUP = max(1-0.4*HSUP/cos(alphaL)/ZSDT(x,LWL),0.67)
        else:
            kSUP = max(1-0.3*HSUP/cos(alphaL)/ZSDT(x,LWL),0.5)
            
    elif Zone == "Flancs des cockpits ouverts":
         
        if ZoneDetail == "Zone ou on marche":
            kSUP = 0.67
        else:
            kSUP = 0.5
        
    elif Zone == "Flancs des cockpits fermes":
         
        if ZoneDetail == "Zone ou on marche":
            kSUP = max(1-0.4*max(hSIDE,3/2*0.500)/cos(alphaL)/ZSDT(x,LWL),0.67)
        else:
            kSUP = max(1-0.3*max(hSIDE,3/2*0.350)/cos(alphaL)/ZSDT(x,LWL),0.5)
            
    elif Zone == "Dessus des superstructures":
         
        if ZoneDetail == "Zone ou on marche":
            kSUP = max(1-0.5*HSUP/ZSDT(x,LWL),0.5)
        else:
            kSUP = max(1-0.5*HSUP/ZSDT(x,LWL),0.35)
            
    elif Zone == "Flancs arrière non protégés des superstructures":
         
        kSUP = max(1-0.6*HSUP/ZSDT(x,LWL),0.35)
        
    elif Zone == "Flancs arrière protégésa des superstructures":
                
        kSUP = max(1-0.7*HSUP/ZSDT(x,LWL),0.35)
        
    if Type == 'Sailing':
        
        # 9.2 Pression de conception pour les voiliers    
        PBSBASE = (2*mLDC**0.33+18)*kSLS
        PDSBASE = 0.5*mLDC**0.33+12
        PBSMINPLT = max((0.3*mLDC**0.33+0.66*LWL*kDC)*kL,10*TC,7)
        PBSMINSTF = max(0.85*PBSMINPLT,7)

        if Zone == "Fond":


            PBSPLT = max(PBSBASE*kAR*kDC*kL,PBSMINPLT)
            PBSSTF = max(PBSBASE*kAR*kDC*kL,PBSMINSTF)
            POUTPLT = PBSPLT
            POUTSTF = PBSSTF

        elif Zone == "Pont et fond de cockpit":

            PDS = (PBSBASE-(PBSBASE-PDSBASE)*min(ZQ/ZSDT(x,LWL),1))*kAR*kDC*kL
            POUTPLT = PDS
            POUTSTF = PDS

        elif Zone == "Muraille":

            PSS = (PBSBASE-(PBSBASE-PDSBASE)*min(ZQ/ZSDT(x,LWL),1))*kAR*kDC*kL
            PSSPLT = max(PBSMINPLT-ZQ/ZSDT(x,LWL)*(PBSMINPLT-5),5)
            PSSSTF = max(0.85*PSSPLT,5)
            POUTPLT = PSSPLT
            POUTSTF = PSSSTF

        else:

            PSS = (PBSBASE-(PBSBASE-PDSBASE)*min(ZSDA/ZSDT(x,LWL),1))*kAR*kDC*kL*kSUP               
            POUTPLT = PSS
            POUTSTF = PSS
    
    elif Type == 'Motor':
    
        # 9.1 Pression de conception pour les bateaux à moteur    
        PBMDBASE = (2.4*mLDC**0.33+20)
        PBMPBASE = 0.1*mldc/LWL/BC*(1+kDC**.5*kDYN)
        PDMBASE = 0.31*mLDC**0.33+12
        PBMMINPLT = max((0.45*mLDC**0.33+0.9*LWL*kDC)*kL,10*TC,7)
        PBMMINSTF = max(0.85*PBSMINPLT,7)

        if Zone == "Fond":
            
            if Mode == 'Displacement':

                PBMPLT = max(PBMDBASE*kAR*kDC*kL,PBMMINPLT)
                PBMSTF = max(PBMDBASE*kAR*kDC*kL,PBMMINSTF)
                POUTPLT = PBMPLT
                POUTSTF = PBMSTF
            
            elif Mode == 'Planning':
                
                PBMPLT = max(PBMPBASE*kAR*kDC*kL,PBMMINPLT)
                PBMSTF = max(PBMPBASE*kAR*kDC*kL,PBMMINSTF)
                POUTPLT = PBMPLT
                POUTSTF = PBMSTF

        elif Zone == "Pont et fond de cockpit":

            PDM = (0.8*PBMDBASE-(0.8*PBMDBASE-PDMBASE)*min(ZQ/ZSDT(x,LWL),1))*kAR*kDC*kL
            POUTPLT = PDM
            POUTSTF = PDM

        elif Zone == "Muraille":
            
            if Mode == 'Displacement':

                PSMD = (0.8*PBMDBASE-(0.8*PBMDBASE-PDMBASE)*min(ZQ/ZSDT(x,LWL),1))*kAR*kDC*kL
                PSMDPLT = max(0.8*PBMMINPLT-(0.8*PBMMINPLT-5)*ZQ/ZSDT(x,LWL),0.9*LWL*KDC,5)
                PSMDSTF = max(0.85*PSMDPLT,5)
                POUTPLT = PSMDPLT
                POUTSTF = PSMDSTF
                
            elif Mode == 'Planning':
                
                PSMP = (0.25*PBMPBASE-(0.25*PBMPBASE-PDMBASE)*min((ZQ-ZC)/(ZSDT(x,LWL)-ZC),1))*kAR*kDC*kL
                PSMPPLT = max(0.25*PBMMINPLT-(0.25*PBMMINPLT-5)*min((ZQ-ZC)/(ZSDT(x,LWL)-ZC)),0.9*LWL*kDC,5)
                PSMPSTF = max(0.85*PSMPPLT,5)
                POUTPLT = PSMPPLT
                POUTSTF = PSMPSTF
                

        else:

            PSM = (0.8*PBMDBASE-(0.8*PBMDBASE-PDMBASE)*min(ZSDA/ZSDT(x,LWL),1))*kAR*kDC*kL*kSUP               
            POUTPLT = PSM
            POUTSTF = PSM
            
        
    titre_DrawingsWindows1.value =  "<font size=3>▌</font>"+"<font size=2>Design Category </font>"+"<font size=3>"+indexCategorie+"</font>"+"    "+"<font size=2>Type of boat </font>"+"<font size=3>"+Type+"</font>"+ "    "+"<font size=2>Mode of navigation </font>"+ "<font size=3>"+Mode+"</font>"


    return kDC,kSLS,kDYN1,kDYN2,kDYN,kAR,kL,POUTPLT,POUTSTF


In [ ]:
# create callback function for centers anchors
def update_point(trace, points, selector):
    
    global regions,scatter,  props
    
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    n = len(c)
    # what out to the minus or plus 1 regarding position in list from 0 to n-1 
    # and the region numbering from 1 to n
    for i in points.point_inds:
        if c[i] == '#bae2be':
                
                c[i] = '#3BA7B3'
                s[i] = 10
                with lol33.batch_update():
                    details0.value = "region <font size=5>"+str(i+1)+"</font> removed from the selection "
                    lol33['data'][i-1]['line']['color']='#3BA7B3'
                    details1.value = "X-Z center position : <font size=5>"+str('{:.1f}'.format(regions[i].centroid[1]))+"</font>mm ,"+"<font size=5>"+str('{:.1f}'.format(-1*regions[i].centroid[0]))+"</font>mm    "
        else:
                if selectmultipleToggle.value == False:
                    foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
                    for j in range(len(foo_indexes)):
                        c[foo_indexes[j]] = '#3BA7B3'
                        s[foo_indexes[j]] = 10
                        with lol33.batch_update():
                            scatter.marker.color = c
                            scatter.marker.size = s
                            lol33['data'][foo_indexes[j]-1]['line']['color']='#3BA7B3'
                c[i] = '#bae2be'
                s[i] = 15
                with lol33.batch_update():
                    details0.value = "region <font size=5>"+str(i+1)+"</font> added to the selection "+"      Major lenght : <font size=5>"+ str('{:.0f}'.format(regions[i].major_axis_length))+"</font>mm"
                    lol33['data'][i-1]['line']['color']='#bae2be'
                    details1.value = "X-Z center position : <font size=5>"+str('{:.1f}'.format(regions[i].centroid[1]))+"</font>mm ,"+"<font size=5>"+str('{:.1f}'.format(-1*regions[i].centroid[0]))+"</font>mm    "
        
    # find selected indices and get region number to display
    foo_indexes = [str(n+1) for n,x in enumerate(c) if x=='#bae2be']
    lst = ''
    for i in range(len(foo_indexes)):
            lst = lst + ' ' + foo_indexes[i] 
    # update graphical output   
    with lol33.batch_update():
        scatter.marker.color = c
        scatter.marker.size = s
        details1.value = details1.value + "<p>Regions selected : <font size=5>" + lst +"</font>"


In [ ]:

## create callback function for uploader            
#def upload_img(change):
#    uploaded_filename = next(iter(uploader.value))
#    content = uploader.value[uploaded_filename]['content']
#    with open('myfile.png', 'wb') as f: f.write(content)
#    with lol.batch_update():
#        details0.value = uploaded_filename+" has been uploaded."
#        lol.data = []
#        lol.update_annotations(visible=False)
        
        
# create callback function for liveaction
def execute_liveaction(self):
    global regions,scatter, props, tree, root, ap, fp, waterlineHeight, borderHeight, chineHeight
    if active_action.value == 'Create segmentation':
        # Get regions and label
        text = io.imread('myfile.png')
        smooth = filters.gaussian(text, 1, preserve_range=True)
        #text_gray = rgb2gray(text) equivalent
        text_gray = np.dot(smooth[...,:3], [0.2989, 0.5870, 0.1140])
        thr = text_gray < 254
        anti = text_gray > 199
        mask = nd.morphology.binary_fill_holes(thr)
        mask = morphology.remove_small_objects(mask, 10)
        mask = morphology.remove_small_holes(mask, 10)
        
        
        lab, n = nd.label(anti*mask)
        # on fixe le nombre maximum de regions
        if n <= 3500:
            regions = regionprops(lab)
            props = regionprops_table(lab, properties=('centroid',
                                                 'orientation',
                                                 'major_axis_length',
                                                 'minor_axis_length'))
            ###Contour detection using built in SCI-KIT library and trace 
            contours = find_contours(lab,1)
            # get the centroids coordinates from region properties
            centers = np.zeros((0, 2))
            for i in range(n):
                
                #also convert the tupples output into np.array
                a = np.asarray(regions[i].centroid).reshape(1,2)
                centers = np.append(centers,a,axis = 0)
                
            with lol33.batch_update():
                
                details0.value = "Found <font size=5> "+str(len(centers))+"</font> regions et <font size=5> "+str(len(contours))+"</font> contours." 
                for index in range(1, lab.max()):
                        
                        label = regions[index].label
                        contour = measure.find_contours(lab == label, 0.5)[0]
                        y, x = contour.T
                        hoverinfo = 'region '+str(label)
                        #for prop_name in properties:
                        #    hoverinfo += f'<b>{prop_name}: {getattr(props[index], prop_name):.2f}</b><br>'
                        lol33.add_trace(go.Scatter(
                            x=x, y=-1*y, name=label,
                            mode='lines',line_color='#3BA7B3' , fill='toself',hovertemplate=hoverinfo, hoveron='points+fills'))
                  
                # add a trace containing the centroids + flip contours Y-axis to match the image Y-axis
                lol33.add_trace(go.Scatter(x = centers[:, 1], y = -1*centers[:, 0],mode = "markers"))
                # scatter is the variable that contains the n regions centroid coordinates
                scatter = lol33.data[n-1]
                colors = ['#3BA7B3'] *n
                scatter.marker.color = colors
                scatter.marker.size = [10] * n
                scatter.on_click(update_point)
                        
        else:
            with lol33.batch_update():
                details0.value = "Error Found. <font size=5> "+str(n)+"</font> regions is too much. Please select another file." 
        
    elif active_action.value =='Get waterline':
        c = list(scatter.marker.color)
        foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
        border = regions[foo_indexes[0]].coords
        # waterline is the upper value of the selected region
        waterlineHeight = min(border[:,0])
        ap = min(border[:,1])
        fp = max(border[:,1])
        #ind = npwhere(border[:,0]==waterline)
        #print(border[ind,:])
        #print(waterline)
        details0.value = "Water line found @Yposition : <font size=5> -"+str(waterlineHeight)+"</font>mm, "
    
    elif active_action.value =='Get deckline':
        c = list(scatter.marker.color)
        foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
        borderHeight = regions[foo_indexes[0]].coords
        ind2 = np.where(borderHeight[:,1] == int(regions[foo_indexes[0]].centroid[1]))
        # deckline is the set composed by the local upper values of each station of the selected region
        details0.value = "Deck line found @Yposition maximum : <font size=5> -"+str(borderHeight[ind2,0].min())+"</font>mm, "
            
    elif active_action.value =='Get chine':
        c = list(scatter.marker.color)
        foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
        chineHeight = regions[foo_indexes[0]].coords
        ind22 = np.where(chineHeight[:,1] == int(regions[foo_indexes[0]].centroid[1]))
        # deckline is the set composed by the local upper values of each station of the selected region
        details0.value = "Chine line found @Yposition maximum : <font size=5> -"+str(chineHeight[ind22,0].min())+"</font>mm, "
    
    elif active_action.value =='Check ISO requirements':
        # find selected region
        c = list(scatter.marker.color)
        foo_indexes = [n for n,x in enumerate(c) if x=='#bae2be']
        border_selected = regions[foo_indexes[0]].coords
        el2 = regions[foo_indexes[0]].major_axis_length
        el3 = regions[foo_indexes[0]].minor_axis_length
        
        # autodetermine scale factor based on Lwl
        scaleFactor = float(value[16].value)/(fp-ap)
        
        # extraction des donnees d'entree
        #mode = modeButton.value
        #typebateau = typeButton.value        
        zone = zoneButton.value
        zone_detail = zonedetailButton.value
                
        # calculate X position in relation with fp
        position = regions[foo_indexes[0]].centroid[1]-ap
        position = position *scaleFactor
        ZQ = -1*regions[foo_indexes[0]].centroid[0]+waterlineHeight
        ZQ = ZQ * scaleFactor
        
        # calculate ZC position in relation with waterlineHeight
        ind22 = np.where(chineHeight[:,1] == int(regions[foo_indexes[0]].centroid[1]))
        ZC = waterlineHeight - chineHeight[ind22,0].min()
        ZC = ZC *scaleFactor
           
        
        # calculate ZSDA position in relation with waterlineHeight
        ind2 = np.where(borderHeight[:,1] == int(regions[foo_indexes[0]].centroid[1]))
        ZSDA = waterlineHeight - borderHeight[ind2,0].min()
        ZSDA = ZSDA *scaleFactor
        
        details0.value = "Following ISO parameters have been determined @XPosition : <font size=5> - "+str('{:.3f}'.format(position))+"</font>m, "+"ZSDA : <font size=5> "+str('{:.1f}'.format(ZSDA))+"</font>m, "+str('{:.1f}'.format(ZSDA))+"</font>m, "+"ZQ : <font size=5> "+str('{:.3f}'.format(ZQ))+"</font>m "+"ZC : <font size=5> "+str('{:.3f}'.format(ZC))+"</font>m " 
        
        #Execute kernel
        res = methode1(position = position,
                       ZQ = ZQ,
                       ZSDA = ZSDA,
                       zone = zone,
                       zone_detail = zone_detail,
                       ZC = ZC,
                       el2 = el2 ,
                       el3 = el3)

        #Dynamic  update
        pressionDisplay.value = "<font size=7>"+str('{:.1f}'.format(res[8]))+"</font>kPa"
        with lol33.batch_update():
            lol33.add_annotation(x =regions[foo_indexes[0]].centroid[1] ,
                               y =-1*regions[foo_indexes[0]].centroid[0]-10 ,
                               text = str('{:.1f}'.format(res[8]))+'kPa',
                               showarrow = False,
                               yshift=0)

In [ ]:
#METHODE 1 ISO START HERE
import xml.etree.ElementTree as ET

tree = ET.parse('OPTIO.iscn')
root = tree.getroot()

dic1 = ["kDC","kSLS","kDYN1","kDYN2","kDYN","kAR","kL","POUTPLT","POUTSTF"]
dic2 = ["Planant","Deplacement"]
dic3 = [
    "Pont et fond de cockpit",
    "Fond",
    "Muraille",
    "Avant des superstructures",
    "Flancs des superstructures",
    "Flancs des cockpits ouverts",
    "Flancs des cockpits fermes",
    "Dessus des superstructures",
    "Flancs arrière non protégés des superstructures",
    "Flancs arrière protégésa des superstructures"]
dic4 = ["Zone ou on marche", "Zone ou on ne marche pas"]
dic5 = ["Moteur","Voilier"]

In [ ]:
# 8.4 Facteur de distribution longitudinale de pression kL
# calcul specifique de kL a une abscisse donnee
def KL(x,kLa,kLb,LWL):
    return min(kLa*x/LWL+kLb,1)

# 6.2.1 Exigences générales    
# calcul specifique de ZSDT a une abscisse donnee
def ZSDT(x,LWL):
    return (0.0286*LWL+0.115)*x/LWL+0.229

def on_value_change(change):
        regions,scatter, props, tree, root, ap, fp, waterlineHeight, borderHeight
        with lol.batch_update():
            active_action.value =='Check ISO requirements'
            execute_liveaction

        

In [ ]:
### Bandeau supérieur
Bandeau = HTML("<font size=7  color=#b39732>12215_ALIVE !</font><font size=3>  by A.C.E. CATTELLIOT ENGINEERING</font>")
Bandeau

In [ ]:
### # create root figure
lol33 = go.FigureWidget()

# create uploader
#uploader2 = FileUpload(multiple=False)

# # create SelectionSlider for guidance
# active_action = widgets.SelectionSlider(
#                 options=['Create segmentation',
#                          'Get waterline',
#                          'Get chine'
#                          'Get deckline',
#                          'Prout',
#                          'Check ISO requirements'],
#                 value='Create segmentation',
#                 description='',
#                 disabled=False,
#                 continuous_update=False,
#                 orientation='horizontal',
#                 readout=True,
#                 layout={'width': 'stretch'},
#                 style = {'value_width': 'max-content'}
# )


# Create taskselectionbutton
active_action = widgets.ToggleButtons(
    options=['Create segmentation',
             'Get waterline',
             'Get chine',
             'Get deckline',
             'Check ISO requirements'],
    description='Tasks:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #layout={'width': 'stretch'},
    #style = {'value_width': 'max-content'},
    style={"button_width": "480px"},
    #tooltips='',
    layout=layoutfull
#      icons=['check'] * 3
)


# create button for active action
active_actionButton = widgets.Button(
    description='Go',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Active Action',
    icon='check' # (FontAwesome names without the `fa-` prefix),
)


selectmultipleToggle = widgets.ToggleButton(
                            value=False,
                            description='SelectMultiple',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Toggle between single and multiple selection',
                            icon='plus' # (FontAwesome names without the `fa-` prefix)
                        )



# # Create mode button
# modeButton = widgets.ToggleButtons(
#     options=dic2,
#     description='Mode de deplacement:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# #    tooltips='',
# #     icons=['check'] * 3
# )
# Create zone button
zoneButton = widgets.ToggleButtons(
    options=dic3,
    description='Emplacement du panneau:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#    tooltips='',
#     icons=['check'] * 3
)
# Create zone detail buttom
zonedetailButton = widgets.ToggleButtons(
    options=dic4,
    description='Detail emplacement:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#    tooltips='',
#     icons=['check'] * 3
)    
# # Create type buttom
# typeButton = widgets.ToggleButtons(
#     options=dic5,
#     description='Type de bateau:',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
# #    tooltips='',
# #     icons=['check'] * 3
# )

#Boat parameters importation
tree = ET.parse('mach40.iscn')
root = tree.getroot()
# ajout et entree du mode de deplacement
indexMode = ET.Element("indexMode")
indexMode.text = ''
root.insert(1,indexMode)

# ajout et entree de la zone de calcul
indexZone = ET.Element("indexZone")
indexZone.text = ''
root.insert(2,indexZone)

#ajout de la categorie de bateau
indexType = ET.Element("indexCategorie")
indexType.text = ''
root.insert(0,indexType)

#ajout du type de bateau
indexType = ET.Element("indexType")
indexType.text = ''
root.insert(0,indexType)

# ajout et entree de la largeur au patin
BC = ET.Element("BC")
BC.text = ''
root.insert(9,BC)

# ajout et entree du tirant d'eau
TC = ET.Element("TC")
TC.text=''
root.insert(9,TC)

 #A MODIFIER ajout et entree du detail de la zone de calcul
indexZoneDetail = ET.Element("indexZoneDetail")
indexZoneDetail.text = ''
root.insert(3,indexZoneDetail)


# creation du menu global des parametres bateau
placeholder  = [widgets.HTML(root[i].tag) for i in range(21)]
value = [widgets.Text(root[i].text,continuous_update=True) for i in range(21)]
left = widgets.VBox(placeholder)
right = widgets.VBox(value)
boatParameterMenu = widgets.HBox([left,right],layout={'width': 'max-content'})

# create output console with initial message
details0 = HTML("<font size=2>Nothing to analyse yet. Please feed informations.</font>")
details1 = HTML("No selection.")
pressionDisplay = widgets.HTML("<font size=7>0</font>kPa")


titre_DrawingsWindows1 = widgets.HTML("<font size=3>▌Visualize and analyse</font>")


# prepare output logic
lol33.layout.hovermode = 'closest'
lol33.update_yaxes(scaleanchor = "x", scaleratio = 1)
lol33.layout.showlegend = False
lol33.layout.plot_bgcolor = 'white'


# observers
###scatter.on_click(update_point)
#uploader2.observe(response, names='value')
active_actionButton.on_click(execute_liveaction)
#modeButton.observe(on_value_change, names='value')
zoneButton.observe(on_value_change, names='value')
zonedetailButton.observe(on_value_change, names='value')
#typeButton.observe(on_value_change, names='value')

# make it happens !

# gather active ation and active action button
#active_actionZone = HBox([uploader,selectmultipleToggle,active_actionButton,active_action,pressionDisplay]) 
active_actionZone = HBox([uploader,selectmultipleToggle,active_action,active_actionButton]) 
ISO = HBox([zoneButton,zonedetailButton],
              layout=Layout(
                                display='flex',
                                flex_flow='column',
                                #border='solid 2px',
                                justify_items='flex-start',
                                align_items='stretch',
                                width='100%'))

menu_12215_ALIVE = VBox([titre_DrawingsWindows1,details0,active_actionZone,lol33,details1,ISO],
                           layout=Layout(
                                display='flex',
                                flex_flow='column',
                                #border='solid 2px',
                                justify_items='flex-start',
                                align_items='stretch',
                                width='100%'
                           ))

tab_contents = []
children = [gendata,menu_12215_ALIVE,igh,boatParameterMenu]
tab = widgets.Tab(display='flex',
                  flex_flow='row wrap',
                  #border='solid 2px',
                  justify_content='space-around',
                  align_items='flex-start',
                  align_content='flex-start',
                  width='100%')

tab.children = children
tab.set_title(0, 'Description')
tab.set_title(1,  'Drawings')
tab.set_title(2,  'Materials')
tab.set_title(3,  'All the parameters')
tab










#HBox([menu_12215_ALIVE,boatParameterMenu],
#    layout=Layout(
#                    display='flex',
#                    flex_flow='row wrap',
#                    #border='solid 2px',
#                    justify_content='space-around',
#                    align_items='flex-start',
#                    align_content='flex-start',
#                    width='100%'))



In [ ]:
#copyright section
copyright = widgets.HTML("<font size=1>© A.C.E. CATTELLIOT ENGINEERING 2021</font>")
copyright

In [ ]:
titre_DrawingsWindows1.value

In [ ]:
#A.8.2 Facteurs d’ajustement de l’épaisseur pour le bordé

def k2(dim1,dim2,orientation):
    
    # invalid character '−' (U+2212 in the standard
    
    
    if orientation == 'b':
        
        if dim1/dim2 <= 2:
            a = -0.0103
            b = 0.0790
            c = -0.1441
            d = -0.2780
            e = 1.1645
            f = -0.5065
            k2 = a * (dim1/dim2)**5 + b *(dim1/dim2)**4 + c * (dim1/dim2)**3 + d * (dim1/dim2)**2 + e * (dim1/dim2) + f
        else:
            k2 = 0.5
            
     
    elif orientation == 'l':
        
         if dim1/dim2 <= 2:
            a = 0.0227
            b = -0.2497
            c = 1.0846
            d = -2.3255
            e = 2.4592
            f = -0.6860
            k2 = a * (dim1/dim2)**5 + b *(dim1/dim2)**4 + c * (dim1/dim2)**3 + d * (dim1/dim2)**2 + e * (dim1/dim2) + f
         
         else:
            k2 = 0.337
        
    
    return k2



def kSH(dim1,dim2,orientation):
    
    # invalid character '−' (U+2212 in the standard
    
    
    if orientation == 'b':
        
        if dim1/dim2 <= 2:
            a = -0.0032
            b = -0.0194
            c = 0.3325
            d = -1.2586
            e = 1.9183
            kSH = a * (dim1/dim2)**5 + b *(dim1/dim2)**4 + c * (dim1/dim2)**3 + d * (dim1/dim2)**2 + e * (dim1/dim2) + f
        else:
            kSH = 0.52
            
     
    elif orientation == 'l':
        
         if dim1/dim2 <= 2:
            a = 0.0372
            b = -0.3841
            c = 1.5530
            d = -3.0725
            e = 2.9792
            f = -0.6767
            kSH = a * (dim1/dim2)**5 + b *(dim1/dim2)**4 + c * (dim1/dim2)**3 + d * (dim1/dim2)**2 + e * (dim1/dim2) + f
         
         else:
            kSH = 0.46
        
    
    return kSH


#A.8.2.2Facteursdecorrectiondecourbure

def kc(dim1,dim2,c1,c2):
    
    cll = c1/dim1
    cbb = c2/dim2
    
    if cll < 0.03:
        cll = 0.03
    if cbb < 0.03:
        cbb = 0.03
           
    a = np.array([-59.161,-52.061,-42.596,-35.496])
    b = np.array([34.928,30.737,25.148,20.957])
    c = np.array([-7.9717,-7.0151,-5.7396,-4.783])
    d = np.array([1.209,1.0945,0.9569,0.8474])

    #cbsurb = np.array([0.030,0.050,0.075,0.1,0.125,0.150,0.175,0.2,0.225])
    clsurl = np.array([0.030,0.060,0.080,0.1])

    fai = interpolate.interp1d(clsurl , a,kind='cubic')
    fbi = interpolate.interp1d(clsurl , b,kind='cubic')
    fci = interpolate.interp1d(clsurl , c,kind='cubic')
    fdi = interpolate.interp1d(clsurl , d,kind='cubic')
       
    ai = fai(cll)
    bi = fbi(cll)
    ci = fci(cll)
    di = fdi(cll)
   
    kc = ai*(cbb)**3 + bi*(cbb)**2 + ci*(cbb) + di

    # debug
    # xnew = np.arange(0.03, 0.1, 0.01)
    # ynew = fci(xnew)   # use interpolation function returned by `interp1d`
    # plt.plot(clsurl, c, 'o', xnew, ynew, '-')
    # plt.show()

    #cbb = np.arange(0.03, 0.225, 0.01)
    
    #ai = fai(0.045)
    #bi = fbi(0.045)
    #ci = fci(0.045)
    #di = fdi(0.045)

    #kc = ai*(cbsurb)**3 + bi*(cbsurb)**2 + ci*(cbsurb) + di
    #kcnew = ai*(cbb)**3 + bi*(cbb)**2 + ci*(cbb) + di
    #plt.plot(cbsurb, kc, 'o',cbb,kcnew,'-')
    #plt.show()
    
    return kc